# Solution
## Step 1: Create dictionary
- Load the vocab file into numpy using readlines
## Step 2: Create paragraph vectors
- Load all the trainign and test data into an array. This will create 4x12500 length arrays.
 - train/pos
 - train/neg
 - test/pos
 - test/neg
 each of these arays in fact includes a paragraph
## Step 3: Cteate BOW
- tokenize each element in each array. This shanges shape of the vectors. Initially vector shapes are (12500, 1). 
- do wordcount (word, count)
- get rid of most common words
- 


In [67]:
os.chdir('/home/ubuntu/CyrusProjects/mxnet-notebooks/python/tutorials/')

In [68]:
# imports and env-variable
# imports and env-variable
import numpy as np
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
import urllib3
import tarfile
from nltk.corpus import stopwords

#BASE_DIR = '/home/ubuntu/CyrusProjects/mxnet-notebooks/python/tutorials'
BASE_DIR = os.getcwd()
PATH='/aclImdb'

In [4]:
os.system('pip3 install nltk')
nltk.download('all')




[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/ubuntu/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

True

In [5]:
http = urllib3.PoolManager()
url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
path = os.getcwd() + '/local_aclImdb_v1.tar.gz'
chunk_size = 1024

r = http.request('GET', url, preload_content=False)

with open(path, 'wb') as out:
    while True:
        data = r.read(chunk_size)
        if not data:
            break
        out.write(data)

r.release_conn()

In [86]:
tar = tarfile.open("aclImdb_v1.tar.gz")
tar.extractall()
tar.close()

In [6]:
with open('aclImdb/imdb.vocab') as vocab_file:
    vocab_array = vocab_file.readlines()
vocab_array = [x.strip() for x in vocab_array]
print(len(vocab_array))


89527


In [69]:
with open(BASE_DIR + PATH + '/imdb.vocab') as f:
    dic = f.readlines()
dic= [x.strip() for x in dic]
dic = np.array(np.sort(dic))
dic = np.unique(dic)
dic = dic.tolist()
filtered_dic = [word for word in dic if word not in stopwords.words('english')]
print(len(dic))
print(len(filtered_dic))

89527
89380


In [8]:
def sentence_array_creator(path, root_dir):
    os.chdir(path)
    sentences = []
    for file in list(glob.glob("*.txt")):
        with open(file, 'r') as f:
            sentences.append(f.readline().strip().lower())
    return sentences
    os.chdir(root_dir)



In [9]:
os.chdir(BASE_DIR)
root_dir = os.getcwd()

base_dir = BASE_DIR + PATH
pos_train_path = base_dir + '/train/pos'
neg_train_path = base_dir + '/train/neg'
pos_test_path = base_dir + '/test/pos'
neg_test_path = base_dir + '/test/neg'


pos_train_sentences = sentence_array_creator(pos_train_path, root_dir)
neg_train_sentences = sentence_array_creator(neg_train_path, root_dir)
pos_test_sentences = sentence_array_creator(pos_test_path, root_dir)
neg_test_sentences = sentence_array_creator(neg_test_path, root_dir)
print('{}; {}; {}; {}; '.format)

print("{}, {}, {}, {}".format(len(pos_train_sentences), len(neg_train_sentences), len(pos_test_sentences), len(neg_test_sentences)))



<built-in method format of str object at 0x7fe844decbb8>
12500, 12500, 12500, 12500


In [10]:
def get_dic_index(word, dic):
    try:
        ret_val = dic.index(word)
    except ValueError: 
        ret_val = -1
    return ret_val

    
word = "is"
print(get_dic_index(word, dic))
    

39813


In [71]:
def sentence_encoder(sentences, vocab):
    bow = []
    t = RegexpTokenizer(r'\w+')
    prog = 0
    for sentence in sentences:
        if prog % 10 == 0:
            print(prog, sep=' ', end='-',flush=True)
        s = ''.join(sentence)
        tokens = t.tokenize(s)
        indexes = []
        for token in tokens:
            idx = get_dic_index(token, vocab)
            if idx != -1:
                indexes.append(idx)
        prog += 1
        bow.append(indexes)
    return bow

def sentence_tokenizer(sentences):
    bow = []
    t = RegexpTokenizer(r'\w+')
    for sentence in sentences:
        s = ''.join(sentence)
        tokens = t.tokenize(s)
        bow.append(tokens)
    return bow


In [12]:
def print_items(array):
    for i in array:
        print('{} \n'.format(i))
        




In [ ]:
print("now calculating bow_pos_train_sentences...")
bow_pos_train_sentences = sentence_encoder(pos_train_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_pos_train_sentences_batch', bow_pos_train_sentences)

print("now calculating bow_neg_train_sentences...")
bow_neg_train_sentences = sentence_encoder(neg_train_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_neg_train_sentences_batch', bow_neg_train_sentences)

print("now calculating bow_pos_test_sentences...")
bow_pos_test_sentences = sentence_encoder(pos_test_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_pos_test_sentences_batch', bow_pos_test_sentences)

print("now calculating bow_neg_test_sentences...")
bow_neg_test_sentences = sentence_encoder(neg_test_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_neg_test_sentences_batch', bow_neg_test_sentences)

print("DONE...")

print(len(bow_neg_test_sentences))
print(len(bow_neg_train_sentences))
print(len(bow_pos_test_sentences))
print(len(bow_pos_train_sentences))


now calculating bow_pos_train_sentences...
0-10-20-30-40-50-60-70-80-90-100-110-120-130-140-150-160-170-180-190-200-210-220-230-240-250-260-270-280-290-300-310-320-330-340-350-360-370-380-390-400-410-420-430-440-450-460-470-480-490-500-510-520-530-540-550-560-570-580-590-600-610-620-630-640-650-660-670-680-690-700-710-720-730-740-750-760-770-780-790-800-810-820-830-840-850-860-870-880-890-900-910-920-930-940-950-960-970-980-990-1000-1010-1020-1030-1040-1050-1060-1070-1080-1090-1100-1110-1120-1130-1140-1150-1160-1170-1180-1190-1200-1210-1220-1230-1240-1250-1260-1270-1280-1290-1300-1310-1320-1330-1340-1350-1360-1370-1380-1390-1400-1410-1420-1430-1440-1450-1460-1470-1480-1490-1500-1510-1520-1530-1540-1550-1560-1570-1580-1590-

In [47]:
a = np.load(BASE_DIR + PATH + '/bow_pos_train_sentences_batch.npy')
b = np.load(BASE_DIR + PATH + '/bow_neg_train_sentences_batch.npy')
c = np.load(BASE_DIR + PATH + '/bow_pos_test_sentences_batch.npy')
d = np.load(BASE_DIR + PATH + '/bow_neg_test_sentences_batch.npy')

print('{}, {}, {}, {}'.format(len(a), len(b), len(c), len(d)))

12500, 12500, 12500, 12500


In [60]:
for item in a:
    a_cnt = (np.bincount(item))
    # print('{}; {}; {}; {}; {}'.format(a_cnt, len(item), len(a_cnt), a_cnt.mean(), a_cnt.var()))
k = [95, 23, 123, 95, 23, 456, 222, 95, 123, 95, 123, 657]
#print(np.bincount(k)


x = a[100]
y = np.bincount(x)
ii = np.nonzero(y)[0]    
zip(ii,y[ii]) 
print(np.vstack((ii,y[ii])).T)
print(dic[80083])

[[   31     7]
 [  253     1]
 [  888     1]
 [ 1211     1]
 [ 1252     1]
 [ 1713     1]
 [ 1830     2]
 [ 2027     2]
 [ 2108     2]
 [ 2195     1]
 [ 2518     1]
 [ 2614    14]
 [ 3281     1]
 [ 3292     1]
 [ 3294     1]
 [ 3554     1]
 [ 3728     2]
 [ 3837     1]
 [ 3917     1]
 [ 4108     2]
 [ 4424     1]
 [ 4983     1]
 [ 5255     1]
 [ 5918     1]
 [ 6333     1]
 [ 6413     1]
 [ 6474     1]
 [ 6576     1]
 [ 6617     1]
 [ 7157     1]
 [ 7504     1]
 [ 9132     1]
 [ 9147    20]
 [ 9973     1]
 [10451     1]
 [10492     8]
 [10589     1]
 [11048     1]
 [11308     1]
 [11377     2]
 [11958     1]
 [14735     1]
 [14873     1]
 [14997     1]
 [15090     1]
 [15480     1]
 [15793     1]
 [16902     1]
 [17676     1]
 [17868     1]
 [17898     3]
 [18256     1]
 [18257     1]
 [18421     1]
 [18962     1]
 [20233     1]
 [20651     1]
 [20778     1]
 [21482     1]
 [21486     2]
 [21648     1]
 [23299     1]
 [23899     1]
 [24282     1]
 [24750     1]
 [25003     1]
 [25171   

89527
89380


In [210]:
def create_bag_of_words(input_para):
    output_para = []
    
    
    return output_para

4600 in 140033611142912 
4600 in 140033602750208 




In [70]:
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [38]:
d = StringIO("M 21 72\nF 35 58")
np.loadtxt(d, dtype={'names': ('gender', 'age', 'weight'),'formats': ('S1', 'i4', 'f4')})

NameError: name 'StringIO' is not defined

In [17]:
from keras.datasets import imdb

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Using TensorFlow backend.


Loading data...
25000 train sequences
25000 test sequences


In [19]:
import lstm_bucketing


    train_sent, vocab = tokenize_text("./data/ptb.train.txt", start_label=start_label,
                                      invalid_label=invalid_label)
    val_sent, _ = tokenize_text("./data/ptb.test.txt", vocab=vocab, start_label=start_label,
                                invalid_label=invalid_label)

    data_train  = mx.rnn.BucketSentenceIter(train_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label)
    data_val    = mx.rnn.BucketSentenceIter(val_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label)


ImportError: No module named 'lstm_bucketing'